In [6]:
from random import shuffle
import glob
shuffle_data = True  # shuffle the addresses before saving
hdf5_path = '../data/banana_vs_avocado.hdf5'  # address to where you want to save the hdf5 file
cat_dog_train_path = '../data/*/*.jpg'
# read addresses and labels from the 'train' folder
addrs = glob.glob(cat_dog_train_path, recursive=True)
labels = [0 if 'Banana' in addr else 1 for addr in addrs]  # 0 = Cat, 1 = Dog
len(labels)

917

In [7]:
# to shuffle data
if shuffle_data:
    c = list(zip(addrs, labels))
    shuffle(c)
    addrs, labels = zip(*c)
    
# Divide the hata into 80% train, 20% validation
train_addrs = addrs[0:int(0.8*len(addrs))]
train_labels = labels[0:int(0.8*len(labels))]
val_addrs = addrs[int(0.8*len(addrs)):]
val_labels = labels[int(0.8*len(labels)):]

In [10]:
import numpy as np
import h5py
dt = h5py.special_dtype(vlen=str)     # PY3

data_order = 'tf'  # 'th' for Theano, 'tf' for Tensorflow
# check the order of data and chose proper data shape to save images
if data_order == 'th':
    train_shape = (len(train_addrs), 3, 224, 224)
    val_shape = (len(val_addrs), 3, 224, 224)
elif data_order == 'tf':
    train_shape = (len(train_addrs), 224, 224, 3)
    val_shape = (len(val_addrs), 224, 224, 3)
# open a hdf5 file and create earrays
hdf5_file = h5py.File(hdf5_path, mode='w')
hdf5_file.create_dataset("train_set_x", train_shape, np.int8)
hdf5_file.create_dataset("list_classes", (2,), dtype=dt)
hdf5_file.create_dataset("val_set_x", val_shape, np.int8)
hdf5_file.create_dataset("train_mean", train_shape[1:], np.float32)
hdf5_file.create_dataset("train_set_y", (len(train_addrs),), np.int8)
hdf5_file["train_set_y"][...] = train_labels
hdf5_file.create_dataset("val_set_y", (len(val_addrs),), np.int8)
hdf5_file["val_set_y"][...] = val_labels
hdf5_file["list_classes"][...] = ['banana', 'bacate bonito']


In [11]:
import cv2
# a numpy array to save the mean of the images
mean = np.zeros(train_shape[1:], np.float32)
# loop over train addresses
for i in range(len(train_addrs)):
    # print how many images are saved every 1000 images
    if i % 10 == 0 and i > 1:
        print ('Train data: {}/{}'.format(i, len(train_addrs)))
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    addr = train_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # add any image pre-processing here
    # if the data order is Theano, axis orders should change
    if data_order == 'th':
        img = np.rollaxis(img, 2)
    # save the image and calculate the mean so far
    hdf5_file["train_set_x"][i, ...] = img[None]
    mean += img / float(len(train_labels))
# loop over validation addresses
for i in range(len(val_addrs)):
    # print how many images are saved every 1000 images
    if i % 10 == 0 and i > 1:
        print ('Validation data: {}/{}'.format(i, len(val_addrs)))
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    addr = val_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # add any image pre-processing here
    # if the data order is Theano, axis orders should change
    if data_order == 'th':
        img = np.rollaxis(img, 2)
    # save the image
    hdf5_file["val_set_x"][i, ...] = img[None]

# save the mean and close the hdf5 file
hdf5_file["train_mean"][...] = mean
hdf5_file.close()

Train data: 10/733
Train data: 20/733
Train data: 30/733
Train data: 40/733
Train data: 50/733
Train data: 60/733
Train data: 70/733
Train data: 80/733
Train data: 90/733
Train data: 100/733
Train data: 110/733
Train data: 120/733
Train data: 130/733
Train data: 140/733
Train data: 150/733
Train data: 160/733
Train data: 170/733
Train data: 180/733
Train data: 190/733
Train data: 200/733
Train data: 210/733
Train data: 220/733
Train data: 230/733
Train data: 240/733
Train data: 250/733
Train data: 260/733
Train data: 270/733
Train data: 280/733
Train data: 290/733
Train data: 300/733
Train data: 310/733
Train data: 320/733
Train data: 330/733
Train data: 340/733
Train data: 350/733
Train data: 360/733
Train data: 370/733
Train data: 380/733
Train data: 390/733
Train data: 400/733
Train data: 410/733
Train data: 420/733
Train data: 430/733
Train data: 440/733
Train data: 450/733
Train data: 460/733
Train data: 470/733
Train data: 480/733
Train data: 490/733
Train data: 500/733
Train dat

In [12]:
subtract_mean = False
# open the hdf5 file
hdf5_file = h5py.File(hdf5_path, "r")
# subtract the training mean
if subtract_mean:
    mm = hdf5_file["train_mean"][0, ...]
    mm = mm[np.newaxis, ...]
# Total number of samples
data_num = hdf5_file["train_set_x"].shape[0]
data_num

733

In [13]:
from random import shuffle
from math import ceil
import matplotlib.pyplot as plt
batch_size = 10
nb_class=2
# create list of batches to shuffle the data
batches_list = list(range(int(ceil(float(data_num) / batch_size))))
shuffle(batches_list)
# loop over batches
for n, i in enumerate(batches_list):
    i_s = i * batch_size  # index of the first image in this batch
    i_e = min([(i + 1) * batch_size, data_num])  # index of the last image in this batch
    # read batch images and remove training mean
    images = hdf5_file["train_set_x"][i_s:i_e, ...]
    
    if subtract_mean:
        images -= mm
    # read labels and convert to one hot encoding
    labels = hdf5_file["train_set_y"][i_s:i_e]
    labels_one_hot = np.zeros((batch_size, nb_class))
    labels_one_hot[np.arange(batch_size), labels] = 1
    print (n+1, '/', len(batches_list))
    print (labels[0], labels_one_hot[0, :])
    plt.imshow(images[0])
    plt.show()
    if n == 5:  # break after 5 batches
        break
hdf5_file.close()

1 / 74
0 [1. 0.]


<Figure size 640x480 with 1 Axes>

2 / 74
1 [0. 1.]


<Figure size 640x480 with 1 Axes>

3 / 74
0 [1. 0.]


...

<Figure size 640x480 with 1 Axes>

5 / 74
1 [0. 1.]


<Figure size 640x480 with 1 Axes>

6 / 74
1 [0. 1.]


<Figure size 640x480 with 1 Axes>